# Working with Gaia data in Python

*This notebook draws heavily on material from Allen Downey's excellent Astronomical Data in Python course materials, which can be accessed [here](https://allendowney.github.io/AstronomicalData/README.html).*

This notebook will introduce you to working with data from the Gaia spacecraft. Though Gaia data can be searched and retrieved from [ESA's Gaia Archive](https://gea.esac.esa.int/archive/), it's often more convenient to do this with Python. For example, if you're systematically retrieving Gaia observations for several different patches of sky, you might want to write a script to carry out the retrieval automatically rather than having to perform the searches one-by-one by hand.

The general workflow for selecting and downloading data from the Gaia Archive is:

1. Make a connection to the Gaia server

2. Explore information about the database and the tables it contains (note that this is not always necessary, and often easier on the Gaia Archive website)

3. Write a query and send it to the server

4. Download the response from the server

5. Continue on with your analysis!

## Connecting to Gaia

The library we'll use to get Gaia data is called <code>astroquery</code> (the documentation can be found [here](https://astroquery.readthedocs.io/en/latest/)).
Specifically, we'll use `Gaia`, which is an [object that represents a connection to the Gaia database](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html).

We can connect to the Gaia database like this: 

In [ ]:
from astroquery.gaia import Gaia

This import statement creates a [TAP+](http://www.ivoa.net/documents/TAP/) connection; TAP stands for "Table Access Protocol", which is a network protocol for sending queries to the database and getting back the results. 

You can include this import statement anywhere in your code, as long as it's run before you try to use `Gaia`. Common practice is to put it with any other import statements at the top of your notebook.

If you encountered a <code>ModuleNotFoundError</code> when running the above cell, you probably need to install <code>astroquery</code> in your environment. Uncomment the <code>pip install</code> line in the next cell, run it, and then delete the cell (or comment out the line again so you don't accidentally rerun it). Then try to run the import cell again.

In [ ]:
#!pip install astroquery

You're now able to query Gaia's databases! 

For certain queries, you might also want to log in to your Gaia Archive account. You can still use <code>astroquery</code> to access Gaia data without being logged in, but the length of time your queries can run for and the number of sources you can retrieve will both be limited. These limitations won't apply if you log in to your account.

To log in, simply run the cell below and enter your username and password. Note that your login can time out! If you leave your notebook running for a while and aren't able to run Gaia queries when you come back, you probably just have to log in again (which is as simple as rerunning this cell).

In [ ]:
#Gaia.login()

## Exploring the database

We can use `Gaia.load_tables` to get the names of the tables in the Gaia database.  With the option `only_names=True`, it loads information about the tables, called "metadata", not the data itself. This is desired because the tables themselves are huge!

In [ ]:
tables = Gaia.load_tables(only_names=True)

In [ ]:
for table in tables:
    print(table.name)

That's a lot of tables! Everything is described in the Gaia documentation (see [here](https://gea.esac.esa.int/archive/documentation/GDR3/) for the DR3 documentation). Most of the time, though, we just want to use the main Gaia table for our data release of interest. In this notebook, we'll stick to the main table for DR3, which is called <code>gaiadr3.gaia_source</code>.

We can use `load_table` (not `load_tables`) to get the metadata for a single table.  The name of this function is misleading, because it only downloads metadata, not the contents of the table.

In [ ]:
meta = Gaia.load_table('gaiadr3.gaia_source')
meta

Jupyter shows that the result is an object of type `TapTableMeta`, but it does not display the contents.

To see the metadata, we have to print the object.

In [ ]:
print(meta)

We can examine the columns in this table using the `.columns` attribute of the `meta` object:

In [ ]:
for column in meta.columns:
    print(column.name)

You can probably infer what many of these columns are by looking at the names, but you should resist the temptation to guess. To find out what the columns mean, [read the documentation](https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_gaia_source.html#gaia_source-).

### Exercise

Gaia provides a stripped-down version of the main table for DR3, called `gaiadr3.gaia_source_lite`. Use `load_table` to get the metadata for this table.  How many columns are in the table, and how does this compare to `gaiadr3.gaia_source`? 

In [ ]:
#Put your solution here!

## Writing queries

With tables as big as those stored in the Gaia database, you generally don't download the entire thing. Instead, you use queries to select only the data you want. A query is a string written in a query language (like [SQL](https://en.wikipedia.org/wiki/SQL)); for the Gaia database, the query language is a dialect of SQL called ADQL.

Here's an example of a simple ADQL query:

In [ ]:
query1 = """SELECT 
TOP 10
source_id, ra, dec, parallax 
FROM gaiadr3.gaia_source
"""

**NOTE:** We're using a triple-quoted string here so we can include line breaks in the query, which makes it easier to read.

The words in uppercase are ADQL keywords:

* `SELECT` indicates that we are selecting data (as opposed to adding or modifying data, which you'll NEVER do to an external table).

* `TOP` indicates that we only want the first 10 rows of the table, which is useful for testing a query before asking for all of the data.

* `FROM` specifies which table we want data from.

The third line is a list of column names separated by commas, indicating which columns we want.  

Here, the keywords are capitalized and the column names are lowercase.  This is a common style, but not required.  ADQL and SQL are not case-sensitive.

The query is also broken into multiple lines to make it more readable.  This is a common style, but not required.  Line breaks don't affect the query.

To run this query, we use the `Gaia` object, which represents our connection to the Gaia database, and invoke `launch_job`:

In [ ]:
job = Gaia.launch_job(query1)
job

The result is an object that represents the job running on a Gaia server.

If you print it, it displays metadata for the forthcoming results.

In [ ]:
print(job)

Don't worry about `Results: None`.  That does not actually mean there are no results.

However, `Phase: COMPLETED` indicates that the job is complete, so we can get the results like this:

In [ ]:
results = job.get_results()
type(results)

The `type` function indicates that the result is an Astropy <code>[Table](https://docs.astropy.org/en/stable/table/)</code>. If we just type the name of a table in a cell (or at the bottom of a cell with other code) and run it, Jupyter knows how to display the table:

In [ ]:
results

Each column has a name, units, and a data type.

For example, the units of `ra` and `dec` are degrees, and their data type is `float64`, which is a 64-bit [floating-point number](https://en.wikipedia.org/wiki/Floating-point_arithmetic), used to store measurements with a fraction part.

This information comes from the Gaia database, and has been stored in the Astropy `Table` by Astroquery.

### Exercise

Read [the documentation](https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_gaia_source.html#gaia_source-)  of this table and choose a column that looks interesting to you.  Add the column name to the query and run it again.  What are the units of the column you selected?  What is its data type?

In [ ]:
#Put your solution here!

## Asynchronous queries

`launch_job` asks the server to run the job "synchronously", which normally means it runs immediately.  But synchronous jobs are limited to 2000 rows.  For queries that return more rows, you should run "asynchronously", which mean they might take longer to get started.

(If you are not sure how many rows a query will return, you can use the ADQL command `COUNT` to find out how many rows are in the result without actually returning them. For example, you could replace `TOP 3000 <col_names>` with `COUNT(col_name)` in `query2` below.)

The results of an asynchronous query are stored in a file on the server, so you can start a query and come back later to get the results. For anonymous users, files are kept for three days.

As an example, let's try a query that's similar to `query1`, with these changes:

* It selects the first 3000 rows, so it is bigger than we should run synchronously

* It selects two additional columns, `pmra` and `pmdec`, which are proper motions along the axes of `ra` and `dec`

* It uses a new keyword, `WHERE`

In [ ]:
query2 = """SELECT 
TOP 3000
source_id, ra, dec, pmra, pmdec, parallax
FROM gaiadr3.gaia_source
WHERE parallax < 1
"""

A `WHERE` clause indicates which rows we want; in this case, the query selects only rows "where" `parallax` is less than 1.  This has the effect of selecting stars with relatively low parallax, which are farther away. 

`WHERE` is one of the most common clauses in ADQL/SQL, and one of the most useful, because it allows us to download only the rows we need from the database. You can use a `WHERE` clause to define criteria using any of the rows in a given table, or combinations of those rows.

We use `launch_job_async` to submit an asynchronous query.

In [ ]:
job = Gaia.launch_job_async(query2)
job

In [ ]:
results = job.get_results()
results

You might notice that some values of `parallax` are negative.  As [this FAQ explains](https://www.cosmos.esa.int/web/gaia/archive-tips#negative%20parallax), "Negative parallaxes are caused by errors in the observations."  They have "no physical meaning," but they can be a "useful diagnostic on the quality of the astrometric solution."

### Exercise

The clauses in a query have to be in the right order.  Go back and change the order of the clauses (lines) in `query2` and run it again.
The modified query should fail, but notice that you don't get much useful debugging information.

For this reason, developing and debugging ADQL queries can be really hard.  A few suggestions that might help:

* Whenever possible, start with a working query, either an example you find online or a query you have used in the past.

* Make small changes and test each change before you continue.

* While you are debugging, use `TOP` to limit the number of rows in the result.  That will make each test run faster, which reduces your development time.  

* Launching test queries synchronously might make them start faster, too.

## Operators

In a `WHERE` clause, you can use any of the [SQL comparison operators](https://www.w3schools.com/sql/sql_operators.asp); here are the most common ones:

| Symbol | Operation
|--------| :---
| `>` | greater than
| `<` | less than
| `>=` | greater than or equal
| `<=` | less than or equal
| `=` | equal
| `!=` or `<>` | not equal

Most of these are the same as Python, but some are not.  In particular, notice that the equality operator is `=`, not `==`.
Be careful to keep your Python out of your ADQL!

You can combine comparisons using the logical operators:

* `AND`: True if both comparisons are true
* `OR`: True if either or both comparisons are true

Finally, you can use `NOT` to invert the result of a comparison. 

### Exercise

Modify the previous query to select rows where `bp_rp` is between `-0.75` and `2`. `bp_rp` contains BP-RP color, which is the difference between two other columns, `phot_bp_mean_mag` and `phot_rp_mean_mag`.

In [ ]:
#Put your solution here!

## Formatting queries

The queries we have written so far are string "literals", meaning that the entire string is part of the program.
But writing queries yourself can be slow, repetitive, and error-prone. It's often better to write Python code that assembles a query for you. This is a place where using [f-strings](https://www.geeksforgeeks.org/formatted-string-literals-f-strings-python/) can be very helpful.

As an example, we'll divide the previous query into two parts; a list of column names and a "base" for the query that contains everything except the column names. Here's the list of columns we'll select:

In [ ]:
columns = 'source_id, ra, dec, pmra, pmdec, parallax'

And here's the base; it's a string that contains at least one format specifier in curly brackets (braces).

In [ ]:
query3_base = f"""SELECT 
TOP 10 
{columns}
FROM gaiadr3.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2
"""

This base query contains one format specifier, `{columns}`, which is a placeholder for the list of column names we will provide. By putting a `f` in front of the string, we tell Python to replace `{columns}` with the actual string that we defined previously. Let's see if it works:

In [ ]:
print(query3_base)

In [ ]:
job = Gaia.launch_job(query3_base)
print(job)

In [ ]:
results = job.get_results()
results

It worked! This is a bit of a contrived example, but let's imagine a scenario where you want to write a function that performs your Gaia queries for you. If you made `columns` an argument for your function and included `query3_base` in the body, you could reuse the same base over and over, changing the column names easily just by changing the name you pass into your function.

**NOTE:** In this example, the variable that contains the column names and the variable in the f-string have the same name: `columns`. That's not required, but it's a common style.

### Exercise

This query always selects sources with `parallax` less than 1.  But suppose you want to take that upper bound as an input.

Modify `query3_base` to replace `1` with a format specifier like `{max_parallax}`. Then define a `max_parallax` variable separately from the query, and verify that this information is automatically added to the string.

In [ ]:
#Put your solution here!

## Working with astronomical coordinates and units

Often we want to query Gaia for sources that occupy a specific region of the sky, which means providing coordinates that tell Gaia where in the sky to look. The "default" coordinate system in astronomy is [right ascension and declination (RA and DEC)](http://spiff.rit.edu/classes/phys445/lectures/radec/radec.html). 

We'll also want to specify the size of the region to look at. Astronomers usually use [angular quantities](https://lco.global/spacebook/sky/using-angles-describe-positions-and-apparent-sizes-objects/#:~:text=Astronomers%20use%20angular%20measure%20to,designated%20by%20the%20symbol%20%C2%B0.) for this. The base unit of angular size is the degree. As you probably know, a full circle is divided into 360°. One degree can be divided into 60 arcminutes (abbreviated 60 arcmin or 60'). An arcminute can also be divided into 60 arcseconds (abbreviated 60 arcsec or 60"). 

The measurements we will work with are physical quantities, which means that they have two parts, a value and a unit. For example, the coordinate 30° has value 30 and its units are degrees. Astropy provides tools for including units explicitly in computations, which is extremely useful for ensuring that conversions are done properly and your calculations are returning the right results.

To use Astropy units, you have to import the `astropy.units` subpackage. Convention is to abbreviate this subpackage with `u`:

In [ ]:
import astropy.units as u

`u` is an object that contains most common units and all SI units. You can use `dir` to list them, but you should also [read the documentation](https://docs.astropy.org/en/stable/units/).

In [ ]:
dir(u)

To create a physical quantity, we multiply a value by a unit.

In [ ]:
angle = 10 * u.degree
type(angle)

The result is a `Quantity` object.
Jupyter has special formatting for displaying objects like this:

In [ ]:
angle

Quantities provide a method called `to` that converts to other units.  For example, we can compute the number of arcminutes in `angle`:

In [ ]:
angle_arcmin = angle.to(u.arcmin)
angle_arcmin

If you add quantities that are the same type of measurement (e.g. angular size) but have different units, Astropy converts them to compatible units, if possible:

In [ ]:
angle + 30 * u.arcmin

If the units are not compatible (often because the measurement types are different), you get an error.
For example:

```
angle + 5 * u.second
```

causes a `UnitConversionError`. This is because you're trying to add one quantity in units of angular size and one quantity in units of time, which doesn't make sense! You can't convert angular size into time.

You'll also see an error if you try to add a "dimensionless quantity" (a number without associated units) to a `Quantity` object. To avoid this, it's best to always attach units to ALL of your objects. However, if you want to access just the value of a `Quantity` without the attached units, you can use the `.value` attribute:

In [ ]:
#Throws an error
angle + 30 

In [ ]:
#Acceptable: both quantities are dimensionless
angle.value + 30 

#Acceptable: both quantities have units
angle + 30*u.arcmin

To represent sky coordinates relative to a specified frame, Astropy provides the `SkyCoord` object.

The following example creates a `SkyCoord` object that represents the approximate coordinates of [Betelgeuse](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=Betelgeuse) in the (default) ICRS frame. [ICRS](https://www.iers.org/IERS/EN/Science/ICRS/ICRS.html) is the
"International Celestial Reference System", adopted in 1997 by the International Astronomical Union.

In [ ]:
from astropy.coordinates import SkyCoord

ra = 88.8 * u.degree
dec = 7.4 * u.degree
coord_icrs = SkyCoord(ra, dec)

coord_icrs

`SkyCoord` is a very flexible object (see the [documentation](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) to learn more about it). You can pass in multiple formats of RA and DEC values. You can also use one `SkyCoord` object to store a list of coordinates (rather than just one coordinate), which allows you to use a bunch of useful methods for catalog matching and other functionality.

### Exercise

Create a quantity that represents 5 [arcminutes](https://en.wikipedia.org/wiki/Minute_and_second_of_arc) and assign it to a variable called `radius`.

Then convert it to degrees.

In [ ]:
#Put your solution here!

## Selecting a region

Now that we understand units and coordinates, we can create queries that select particular regions of the sky. For example, here's a query from the [Gaia archive documentation](https://gea.esac.esa.int/archive-help/adql/examples/index.html) that selects objects in a circular region centered at (88.8, 7.4) with a search radius of 5 arcmin (0.08333 deg).

In [ ]:
query_cone = """SELECT 
TOP 100 
source_id, ra, dec
FROM gaiadr3.gaia_source
WHERE 1=CONTAINS(
  POINT(ra, dec),
  CIRCLE(88.8, 7.4, 0.08333333))
"""

This query uses three keywords that are specific to ADQL (not SQL):

* `POINT`: a location in [ICRS coordinates](https://en.wikipedia.org/wiki/International_Celestial_Reference_System), specified in **degrees** of right ascension and declination.

* `CIRCLE`: a circle where the first two values are the coordinates of the center and the third is the radius in degrees.

* `CONTAINS`: a function that returns `1` if a `POINT` is contained in a shape and `0` otherwise.

Here is the [documentation of `CONTAINS`](http://www.ivoa.net/documents/ADQL/20180112/PR-ADQL-2.1-20180112.html#tth_sEc4.2.12).

A query like this is called a cone search because it selects stars in a cone. Once we've defined our query, we run it just like we would run any other job:

In [ ]:
job = Gaia.launch_job(query_cone)
job

In [ ]:
results = job.get_results()
results

### Exercise

Cone searches are a very common tool for astronomers, who often want to identify all of the sources near a known source. Another case you might encounter is wanting to find all of the sources that fall within a certain rectangle on the sky, defined by corners in RA and DEC. You can do this by defining a `POLYGON` in ADQL (see the resources linked at the bottom of this notebook to learn how), or you can simply chain together conditional statements that check for RAs and DECs within your rectangular boundaries. 

Using the latter approach, write a query that finds all sources within a rectangle ranging from 88.3 to 89.3 in RA and 6.9 to 7.9 in DEC.

In [ ]:
#Put your solution here!

## Working with the results of your query

So now that we've retrieved our data, how do we work with it?

As mentioned previously, `astroquery` returns the results of each Gaia query as an Astropy `Table` object. There's a bunch of different ways to [manipulate these objects](https://docs.astropy.org/en/stable/table/indexing.html), but some of the most straightforward commands are outlined below.

You can get the names of the columns in a table like this:

In [ ]:
results.colnames

And select an individual column like this:

In [ ]:
results['ra']

The result is a `Column` object that contains the data, and also the data type, units, and name of the column. Specifically, this is a `MaskedColumn` because there might be missing data values, which are represented by "masked" values. For our purposes, though, the functionality is the same as a normal `Column`.

In [ ]:
type(results['ra'])

The rows in the `Table` are numbered from 0 to `n-1`, where `n` is the number of rows.  We can select the first row like this:

In [ ]:
results[0]

As you might have guessed, the result is a `Row` object.

In [ ]:
type(results[0])

Notice that the bracket operator selects both columns and rows.  You might wonder how it knows which to select.
If the expression in brackets is a string, it selects a column; if the expression is an integer, it selects a row.

If you apply the bracket operator twice, you can select a column and then an element from the column.

In [ ]:
results['ra'][0]

Or you can select a row and then an element from the row.

In [ ]:
results[0]['ra']

You get the same result either way.

If you want to select only a subset of the data in a table, based on the values in one or more of the columns, you can use filtering:

In [ ]:
results[results['ra'] < 88.8]

When you want to chain together multiple filter conditions, put parentheses around each condition, and use `|` for "or", `&` for "and", and `~` for "not".

In [ ]:
results[(results['ra'] < 88.8) & (results['dec'] > 7.34)]

## Saving and loading query results

What if you want to download the results of a Gaia query and save them for later? This is straightforward to do through the online portal, but not as obvious when you run your query from a Python script or notebook. However, it's a useful skill to have. Storing the data in a file means we can shut down this notebook and pick up where we left off without running the previous query again -- this is especially useful for queries that take a long time to run!

Astropy `Table` objects provide `write`, which writes the table to disk.

In [ ]:
filename = 'query_results.fits'
results.write(filename, overwrite=True)

Because the filename ends with `fits`, the table is written in the [FITS format](https://en.wikipedia.org/wiki/FITS), which preserves the metadata associated with the table.

If the file already exists, the `overwrite` argument causes it to be overwritten.

To read the data from the file back into an Astropy `Table`, either in this notebook or in another piece of code, we would then simply use the `.read` method:

In [ ]:
from astropy.table import Table
old_results = Table.read(filename)
old_results

## Putting it all together 

This [Hertzsprung-Russell diagram](https://sci.esa.int/web/gaia/-/60198-gaia-hertzsprung-russell-diagram) shows the BP-RP color and luminosity of stars in the Gaia catalog: 

<img width="300" src="https://github.com/AllenDowney/AstronomicalData/raw/main/images/1567214809100-ESA_Gaia_DR2_HRD_Gaia_625.jpg">

*(Copyright: ESA/Gaia/DPAC, CC BY-SA 3.0 IGO)*

Let's make our own HR diagram with stars that are within 100 parsecs (pc) of the Sun! That distance corresponds to a parallax of 10 mas. To do this:

1. Write a query that selects the TOP 10000 stars that have parallaxes greater than or equal to 10 mas from the main Gaia DR3 table. You'll want the columns listed in the first cell below.

2. Use `matplotlib` to plot a CMD, with BP-RP color on the x-axis and G magnitude on the y-axis. Make sure to invert the y-axis to match the conventions of CMDs!

3. See what features of the Gaia HRD you can identify in your HRD. What's different about your plot? What might be causing the difference?

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
columns = 'source_id, ra, dec, parallax, bp_rp, phot_g_mean_mag'

In [ ]:
#Put your solution here!

## More ADQL resources

The official ADQL documentation can be found [here](https://www.ivoa.net/documents/ADQL/20180112/PR-ADQL-2.1-20180112.html). However, when it comes to writing your own queries, you'll probably find it more helpful to refer to cheatsheets and examples. Here are some of my go-tos:

* [General syntax](https://docs.appdynamics.com/appd/24.x/latest/en/analytics/adql-reference/adql-queries)
* [Simbad ADQL cheatsheet](https://simbad.u-strasbg.fr/simbad/tap/help/adqlHelp.html)
* [Vizier ADQL cheatsheet](https://tapvizier.u-strasbg.fr/adql/help.html)
* [Examples from Gaia](https://gea.esac.esa.int/archive-help/adql/examples/index.html)
* [More examples from Gaia](https://www.cosmos.esa.int/web/gaia-users/archive/writing-queries)